In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, ElasticNetCV, LassoCV, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, RobustScaler, scale
import seaborn as sns 

In [3]:
train_city =  pd.read_csv('data/tract_health/nyc_tract_health.csv')
test_city = pd.read_csv('data/tract_health/atlanta_tract_health.csv')

In [5]:
train_city

,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,TEETHLOST_Crude95CI,Geolocation,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,LowIncomeTracts,PovertyRate,MedianFamilyIncome
0,NY,New York,Kings,36047,36047009600,6126,34.5,"(29.5, 39.2)",15.1,"(14.5, 15.7)",...,"(19.7, 32.7)",POINT (-74.00578419 40.64602128),60.348103,0.158023,0.285906,37.900002,24.892878,1,33.247256,41600.0
1,NY,New York,Kings,36047,36047011600,4816,28.0,"(23.5, 32.6)",16.9,"(16.2, 17.6)",...,"(25.8, 37.9)",POINT (-74.00364817 40.63587202),73.568756,0.273695,0.250513,53.891254,90.054947,1,28.588166,39808.0
2,NY,New York,Kings,36047,36047046000,3729,15.0,"(12.9, 17.3)",19.0,"(18.2, 19.8)",...,"(12.0, 24.8)",POINT (-73.96557391 40.6318131),99.599609,0.216846,0.153766,75.067375,40.882805,1,22.525309,63636.0
3,NY,New York,Bronx,36005,36005031400,1966,13.8,"(11.8, 15.9)",22.0,"(21.0, 22.9)",...,"( 8.1, 16.7)",POINT (-73.84553774 40.8615489),101.599998,0.683035,0.188713,65.904182,18.100000,0,1.491795,117500.0
4,NY,New York,Bronx,36005,36005042300,4100,26.7,"(22.7, 31.0)",16.9,"(16.3, 17.5)",...,"(14.0, 22.7)",POINT (-73.87732222 40.87707259),148.684418,0.000343,0.283845,90.369011,120.772598,1,32.400000,64405.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,NY,New York,Queens,36081,36081113900,3322,13.9,"(11.9, 16.0)",19.7,"(18.9, 20.6)",...,"( 8.0, 17.5)",POINT (-73.79064204 40.76421748),142.859360,0.159934,0.000185,102.975258,46.231773,1,9.969880,57260.0
2113,NY,New York,Kings,36047,36047058000,3365,16.8,"(15.1, 18.5)",18.3,"(17.7, 18.9)",...,"(12.8, 22.6)",POINT (-73.95544864 40.59899814),62.408436,0.286192,0.000097,83.171532,54.999004,1,18.527488,61458.0
2114,NY,New York,New York,36061,36061013900,9257,7.6,"( 6.4, 9.4)",17.3,"(16.5, 18.1)",...,"( 4.4, 10.7)",POINT (-73.98618103 40.76729463),137.177475,0.085202,0.078398,71.648735,33.882988,0,8.772936,129286.0
2115,NY,New York,Queens,36081,36081041400,4033,19.8,"(16.8, 23.0)",20.3,"(19.4, 21.2)",...,"(15.8, 31.4)",POINT (-73.78072667 40.69965768),68.249146,0.307770,0.000254,29.305349,29.014908,1,21.498297,72758.0


In [6]:
test_city

,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,TEETHLOST_Crude95CI,Geolocation,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,LowIncomeTracts,PovertyRate,MedianFamilyIncome
0,GA,Georgia,Fulton,13121,13121010204,4761,9.6,"( 7.5, 12.7)",24.8,"(23.4, 26.2)",...,"( 2.6, 8.2)",POINT (-84.41716366 33.92519129),258.6,1.051088,4.476390,103.4,103.9,0,1.821352,192813.0
1,GA,Georgia,Fulton,13121,13121005501,2307,25.2,"(21.3, 29.6)",20.0,"(18.9, 20.9)",...,"(14.5, 33.7)",POINT (-84.38372162 33.72987572),130.1,0.278257,0.821833,14.1,17.0,1,43.297645,26579.0
2,GA,Georgia,Fulton,13121,13121001202,3937,12.6,"(11.5, 14.2)",10.0,"( 9.6, 10.4)",...,"( 3.4, 6.4)",POINT (-84.38515368 33.77682317),72.2,0.620474,0.266751,37.1,0.0,0,8.592560,133177.0
3,GA,Georgia,Fulton,13121,13121011425,6906,10.8,"( 8.9, 13.5)",20.0,"(18.9, 21.3)",...,"( 2.8, 7.7)",POINT (-84.24164477 34.01149372),289.4,0.963302,3.946356,273.9,111.7,0,2.212389,186010.0
4,GA,Georgia,Fulton,13121,13121009403,4625,18.4,"(15.7, 21.9)",13.5,"(12.9, 14.1)",...,"( 5.9, 17.5)",POINT (-84.35991176 33.83156797),169.2,0.299675,1.285113,195.7,141.4,0,13.447045,68733.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,GA,Georgia,Fulton,13121,13121011623,11473,13.6,"(11.9, 15.4)",16.1,"(15.5, 16.9)",...,"( 3.9, 8.6)",POINT (-84.19032903 34.03959659),289.4,0.963302,3.946356,273.9,111.7,0,2.841046,123885.0
198,GA,Georgia,Fulton,13121,13121000600,5203,20.0,"(17.9, 22.5)",8.4,"( 8.1, 8.7)",...,"(10.3, 17.5)",POINT (-84.40584642 33.78496362),119.0,0.563022,0.387744,54.9,46.4,1,43.860730,96850.0
199,GA,Georgia,Fulton,13121,13121009603,4388,14.2,"(12.2, 17.5)",15.5,"(14.9, 16.2)",...,"( 3.5, 11.6)",POINT (-84.37581823 33.83437332),50.4,0.151147,1.358243,31.6,74.0,0,8.163265,200192.0
200,GA,Georgia,Fulton,13121,13121003900,1331,29.2,"(25.2, 33.5)",26.8,"(25.9, 27.8)",...,"(20.8, 39.1)",POINT (-84.42210552 33.75005537),115.2,0.180219,0.821719,20.4,22.3,1,36.868268,36181.0


In [13]:
name_mapping = {'access2': 'Health insurance access', 'arthritis': 'Arthritis prevalence', 'binge': 'Binge drinking prevalence',
               'bphigh': 'High blood pressure prevalence', 'bpmed': 'Medium blood pressure prevalence', 'cancer': 'Cancer prevalence',
               'casthma': 'Asthma prevalence', 'cervical': 'Cervical cancer screenings', 'chd': 'Coronary heart disease prevalence',
               'checkup': 'Routine checkups', 'cholscreen': 'Cholesterol screenings', 'colon_screen': 'Colon cancer screenings',
               'copd': 'COPD prevalence', 'corem': 'Core men\'s health', 'corew': 'Core women\'s health', 'csmoking': 'Smoking prevalence',
               'dental': 'Dental checkups', 'depression': 'Depression prevalence', 'diabetes': 'Diabetes prevalence', 'ghlth': 'General poor health prevalence',
               'highchol': 'High cholesterol prevalence', 'kidney': 'Chronic kidney disease', 'lpa': 'No physical activity', 'mammouse': 'Mammograms',
               'mhlth': 'Poor mental health prevalence', 'obesity': 'Obesity prevalence', 'phlth': 'Poor physical health', 'sleep': 'Poor sleep prevalence',
               'stroke': 'Stroke prevalence', 'teethlost': 'Teeth loss prevalence'}

results_all_nan = pd.DataFrame({'Health condition': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Public transport': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                        'Education': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Houses of worship': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                       'Poverty Rate': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'RSquared': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 
                            'MSE': [np.nan for c in train_city.columns if c.endswith('CrudePrev')]})
results_nan = pd.DataFrame({'Health condition': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Poverty Rate': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 
                            'RSquared': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 
                            'MSE': [np.nan for c in train_city.columns if c.endswith('CrudePrev')]})
add_vars = ['PovertyRate']

In [14]:
def update_all_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.coef_[3]
        results.iat[i, 5] = regr.coef_[4]
        results.iat[i, 6] = regr.coef_[5]
        results.iat[i, 7] = regr.score(X_test, y_test)
        results.iat[i, 8] = mse_test
def update_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.score(X_test, y_test)
        results.iat[i, 5] = mse_test

In [15]:
#set desert and log transformation params
all_deserts=True
log_transf = False

In [29]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
alphas = []
l1_ratios = []
i=0


for c in train_city.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = train_city[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = train_city[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))
        
        
        xtest = test_city[desert_measures+add_vars].to_numpy(copy=True)
        ytestcity = test_city[c].to_numpy(copy=True)
        scaler = StandardScaler()
        xtestscale = scaler.fit_transform(xtest)
        predictions = regr.predict(xtestscale)
        #mse_test = mean_squared_error(ytestcity, predictions)
        
        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Teeth loss prevalence,0.3279,0.8257,0.3231,-0.0000,-0.000000,6.084550,0.7230,810.540944
1,Poor mental health prevalence,0.1922,0.1788,0.0667,-0.0138,0.050941,2.421954,0.6979,810.540944
2,Poor physical health,0.2394,0.2520,0.2602,-0.0000,0.053062,2.895717,0.6931,810.540944
3,General poor health prevalence,0.4707,0.4852,0.2647,0.0000,-0.000000,6.177815,0.6629,810.540944
4,Smoking prevalence,0.3928,0.4176,0.2193,-0.0000,0.136902,3.166559,0.6530,810.540944
5,Dental checkups,-0.0462,-1.1391,-0.1620,-0.0621,0.021184,-7.982901,0.6236,810.540944
6,No physical activity,0.7006,0.9693,0.5634,0.0000,0.000000,6.230170,0.5670,810.540944
7,Health insurance access,0.0000,0.4070,-0.0034,0.1075,-0.169992,5.477505,0.5304,810.540944
8,Colon cancer screenings,0.0000,-0.6390,-0.0218,-0.0604,-0.000000,-5.288940,0.5303,810.540944
9,COPD prevalence,0.1298,0.2299,0.2870,-0.0784,0.171253,1.327597,0.4891,810.540944


In [26]:
predictions

array([  2.78388691,  -4.05960189,   1.4740756 ,   2.81692965,
         1.55529025,   3.19859432,   1.44254289,   1.66504068,
        -6.30491977,  -1.09953624,   2.26869235,  -4.4929191 ,
         2.44353518,   0.17281604,   3.02443077,   0.51931713,
        -1.72392193,   2.86436356,   1.77938799,  -0.70906139,
         1.99695609,  -3.38468163,   2.48390169,   2.85351891,
         0.08646855,  -1.36869526,  -3.298165  ,  -1.79679418,
         2.3528109 ,  -2.1124572 ,  -2.46959559,  -2.26153412,
        -2.27568488,   2.01793735,   1.72460739,  -0.80307686,
         3.41961474,  -2.46832293,  -1.28551927,   2.36177462,
         1.46589751,   2.64785782,  -2.0426763 ,  -2.89948535,
         0.28446141,   3.0057428 ,  -2.66976309,   2.1979838 ,
        -4.95466196,  -0.02287436,  -4.28309172,   2.24002208,
         1.76646693,   1.23923263,  -0.38155287,   2.7918855 ,
        -0.36453055,   0.86366018,   2.62352621,  -3.10575796,
        -5.97347464,   2.16173272,   2.26548825,  -0.33

In [27]:
ytestcity

array([36.7, 21.1, 35.1, 36.1, 28.5, 32.3, 33. , 28.6, 17.9, 22.7, 37. ,
       18.1, 34.6, 24. , 34. , 29.1, 21.4, 32.9, 34.4, 21.3, 28.6, 21.7,
       34.4, 35.3, 25.9, 26.2, 21.2, 21.1, 31.1, 21. , 20.9, 25.9, 23.4,
       31.2, 33.7, 19.1,  nan, 19.8, 23.3, 33.6, 37. , 28.8, 20.6, 20.9,
       32.6, 34. , 23.6, 33.9, 18.9, 25.5, 19.2, 31. , 35.4, 28.3, 28.9,
       35.9, 19.6, 28.8, 35.1, 18.5, 22.8, 31.2, 34.2, 24.8, 31.5, 33.6,
       33.6, 33.2, 36.4, 34.4, 19.6, 17.6, 27.3, 34.6, 20.9, 32.7, 20. ,
       19.1, 36.1, 34.1, 31.5, 27.5, 37.2, 18.4, 23.3, 19.8, 36.8, 25.7,
       33.3, 17.1, 32.5, 21.9, 14.5, 24.9, 36.3, 22.4, 24.9, 21.3, 29. ,
       36.3, 25.7, 35.2, 35.1, 15.1, 20.2, 21.7, 20.8, 34.3, 25.2, 33.5,
       36.3, 21.9, 29.3, 23.1, 24.7, 17.9, 34.2, 33.9, 34. , 31.3, 29.3,
       33.9, 29. , 30.3, 22. , 33.7, 21.6, 22. , 31.9, 33.3, 29.7, 34.4,
       20. , 32.9, 26.4, 23.3, 26.6, 23.7, 19.1, 36.6, 26. , 33.2, 24.7,
       34.9, 31.1, 36.5, 23. , 22.6, 30.7, 19.6, 27

In [28]:
y

array([-4.90298013, -5.90298013, -0.80298013, ...,  6.39701987,
       -4.10298013,  7.99701987])